Gabriel Marcelino, Grant Burk, and Eli Kaustinen
September 2024
Artificial Neural Network (ANN)

## Import dependencies and load data

In [253]:
import sys
print(sys.path)


['C:\\Program Files\\JetBrains\\PyCharm 2023.3.3\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Program Files\\JetBrains\\PyCharm 2023.3.3\\plugins\\python\\helpers\\pydev', 'C:\\Users\\grant\\PycharmProjects\\neural-networks', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312', '', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\grant\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\setuptools\\_vendor']


In [254]:
import csv
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

pool = []
training_data = []
# create pool with players from 2019-2022
with open('all_seasons.csv', mode = 'r') as file:
    csvFile = csv.reader(file)
    # ignore first line
    next(csvFile)
    for lines in csvFile:
        training_data.append(lines)


## Optimal Team : Considerations
For my optimal team, I will aim for:
- 1 or more players in top 20% of shooting percentage.
- Player in the top 5% out of the 100 for rebounds.
- Player with a Defensive Rebound percentage bigger than 0.2
- 3 or more players in top 20% of best net rating
- 2 or more players with better than average assists

In [255]:
def extract_features(player):
    return {
        'name': player[1],
        'ts_pct': float(player[19]),
        'reb': float(player[13]),
        'dreb_pct': float(player[17]),
        'rating': float(player[15]),
        'ast': float(player[14])
    }


In [256]:
def team_score(team):
    ts_pct = np.mean([player['ts_pct'] for player in team])
    reb = np.sum([player['reb'] for player in team])
    dreb_pct = np.max([player['dreb_pct'] for player in team])
    rating = np.mean([player['rating'] for player in team])
    ast = np.sum([player['ast'] for player in team])
    # For my optimal team, the most important features are
    # 1. True shooting percentage
    # 2. Defensive rebound percentage
    # 3. Rating
    # 4. Assists/Rebounds
    score = (ts_pct * 20 + reb * 0.5 + dreb_pct * 10 + rating + ast * 0.5) / 5
    return score

def select_optimal_team(pool, team_size=5):
    player_features = [extract_features(player) for player in pool]
    
    best_team = None
    best_score = -float('inf')

    # Use a heuristic to find a good solution
    for _ in range(1000):  # Number of iterations
        team = random.sample(player_features, team_size)
        score = team_score(team)
        if score > best_score:
            best_score = score
            best_team = team

    return best_team, best_score





## Model

In [257]:
def extract_features(pool):
    features_list = []
    for player in pool:
        # extract relevant features based on considerations above
        features = {
            'name': player[1],
            'ts_pct': player[19],
            'reb': player[13],
            'dreb_pct': player[17],
            'rating': player[15],
            'ast': player[14]
        }
        features_list.append(features)
    return features_list


## Simulate Training Data to train model

In [258]:
def make_empty(player):
    player['ts_pct'] = 0
    player['reb'] = 0
    player['dreb_pct'] = 0
    player['rating'] = 0
    player['ast'] = 0
    return player

# Get Best Player for Category

In [259]:
def get_best(players, category):
    best_index = 0
    for cur in range(len(players)):
        if float(players[cur][category]) > float(players[best_index][category]):
            best_index = cur
    return best_index

# Pre Process the data

In [260]:
def get_datapool(segment):
    # Make testing data sets
    Xall = extract_features(training_data)
    
    # Create one training set
    X1 = []
    for i in range(100):
        X1.append(Xall[i + segment * 100])
        
    Xval = []
    for player in X1:
        Xval.append(np.array([float(player['ts_pct']), float(player['reb'])/100, float(player['dreb_pct']), float(player['rating'])/100, float(player['ast'])/100]))
    Xval = np.array(Xval)
    
    tempX1 = X1.copy()
    
    # Create empty expected Y array to fill later
    Y1 = np.empty_like(X1)
    for i in range(len(Y1)):
        Y1[i] = 0
    
    # Best ts_pct into Y1
    cur = get_best(tempX1, 'ts_pct')
    tempX1[cur] = make_empty(tempX1[cur]) # Prevent repeat
    Y1[cur] = 1
    
    # Best reb into Y1
    cur = get_best(tempX1, 'reb')
    tempX1[cur] = make_empty(tempX1[cur]) # Prevent repeat
    Y1[cur] = 1
    
    # Best dreb_pct into Y1
    cur = get_best(tempX1, 'dreb_pct')
    tempX1[cur] = make_empty(tempX1[cur]) # Prevent repeat
    Y1[cur] = 1
    
    # Best rating into Y1
    cur = get_best(tempX1, 'rating')
    tempX1[cur] = make_empty(tempX1[cur]) # Prevent repeat
    Y1[cur] = 1
    
    # Best ast into Y1
    cur = get_best(tempX1, 'ast')
    tempX1[cur] = make_empty(tempX1[cur]) # Prevent repeat
    Y1[cur] = 1
    
    Y1 = np.array(Y1)
    
    print()
    print("Xval")
    print(len(Xval))
    print(Xval)
    
    Xvaltemp =  tf.convert_to_tensor(Xval, dtype=tf.float64)
    print()
    print("Xvaltemp")
    print(len(Xvaltemp))
    print(Xvaltemp)
    
    return X1, Xval, Y1 # return raw X (X1), processed X (Xval), and Y value (Y1

X_raw_train, X_train, y_train = get_datapool(1)
X_raw_test, X_test, y_test = get_datapool(2)


Xval
100
[[ 5.44e-01  6.20e-02  1.57e-01  4.50e-02  2.50e-02]
 [ 4.86e-01  2.20e-02  1.55e-01 -7.30e-02  6.00e-03]
 [ 6.45e-01  4.00e-02  1.06e-01 -4.90e-02  4.10e-02]
 [ 4.86e-01  1.70e-02  8.80e-02 -1.14e-01  1.60e-02]
 [ 5.54e-01  1.03e-01  2.07e-01  3.40e-02  4.60e-02]
 [ 5.66e-01  1.30e-02  9.30e-02  2.00e-02  2.20e-02]
 [ 5.62e-01  8.80e-02  1.86e-01  1.10e-01  2.70e-02]
 [ 5.52e-01  2.10e-02  1.77e-01 -5.90e-02  4.00e-03]
 [ 5.80e-01  7.90e-02  2.16e-01 -4.10e-02  6.00e-03]
 [ 5.37e-01  8.30e-02  1.72e-01 -8.30e-02  1.70e-02]
 [ 4.71e-01  5.60e-02  2.23e-01 -6.30e-02  7.00e-03]
 [ 5.48e-01  6.00e-02  1.22e-01  7.50e-02  5.70e-02]
 [ 5.59e-01  4.80e-02  1.82e-01  2.00e-02  6.00e-03]
 [ 5.35e-01  4.10e-02  1.12e-01 -4.20e-02  1.60e-02]
 [ 5.28e-01  1.50e-02  8.10e-02 -8.80e-02  3.20e-02]
 [ 4.12e-01  1.80e-02  1.07e-01  5.70e-02  1.10e-02]
 [ 0.00e+00  3.00e-03  0.00e+00 -1.21e-01  3.00e-03]
 [ 5.32e-01  9.70e-02  2.15e-01 -2.00e-03  7.00e-03]
 [ 5.37e-01  3.60e-02  7.40e-02 -9.1

## Train Model
Now that we have the training data, we can train the model.

In [261]:
# 3. Neural Network Model
model = tf.keras.models.Sequential([
    # Input Layer (implicitly present by specifying input_shape)
    
    # Hidden Layer 1
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    
    # Hidden Layer 2
    tf.keras.layers.Dense(64, activation='relu'),
    
    # Output Layer
    tf.keras.layers.Dense(2, activation='softmax')  # Two categories
])

# 4. Compile the Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 5. Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

## Explain your architecture and how the basketball player characteristics are used as inputs:

1. Data Preparation:

- Pool and Training Data:
    - Separates data: 
        - `pool` holds 100 unique players (2019-2022) for potential future use.
        - `training_data` stores up to 5000 players (2019-2022) for model training.

2. Feature Extraction:

- The `extract_features` function takes a list of players and creates a dictionary of features for each one. 
- These features include:
    - Name (not used as input)
    - True Shooting Percentage (ts_pct)
    - Rebounds (reb)
    - Defensive Rebound Percentage (dreb_pct)
    - Net Rating (rating)
    - Assists (ast)

3. Simulating Data for Training:

- The `simulate_data` function generates training data with labels indicating successful teams based on pre-defined criteria.
- It takes a list of players (`sample`) and a number of simulations (`num_iter`).
- Here's how player characteristics are used as inputs:
    - Averages and percentiles are calculated for assists, shooting percentage, rebounds, and net rating from the `sample` data.
    - Loops through `num_iter` simulations:
        - Selects 5 random players.
        - Extracts their features using `extract_features`.
        - Assigns a label (0 or 1) based on the criteria defined above..

4. Model Building and Training:

TO-DO
5. Evaluation:

TO-DO


## Interpret the output of your MLP in the context of selecting an optimal basketball team:

This MLP is trained on the 'optimal' prediction, given the input features of the dataset-shooting percentage, rebounds, assists, and player ratings amongst others-a basketball lineup is optimal. These mentioned inputs provide key indications about player performance. The model will process it for a binary outcome; 1 stands for an optimum team, and 0 stands for a suboptimum team. It could be shown that the MLP model is currently trained to learn from historical data to understand whether a combination of players is strong or weak and thus provide insight into which lineups will perform likely perform.

These could then inform the choice of an optimum basketball team, where the model shows which sets of players are statistically balanced in key performance metrics. The teams that are predicted as "optimal" will indicate that their set of players is effective, while those labeled "suboptimal" need adjustment. Coaches or team selectors would be able to play around with different mixes of players in order to fine-tune their strategy and ensure the team is great in a lot of aspects of the game: scoring, defense, teamwork, just to name a few.

Besides the precision, recall is one of the metrics that signifies model reliability or accuracy. High accuracy means the model can discriminate between good and bad lineups well. While precision and recall give an indication of few false predictions or missing optimal teams, in the end, what an MLP model provides is a data-driven method of selection of the team so as to best ensure that the lineup chosen will perform better on the court.